<a href="https://colab.research.google.com/github/brockbladebruno-alt/Brock/blob/main/Shooting_Video_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import streamlit as st
import tempfile
import os
from sklearn.linear_model import LinearRegression



def calculate_angle(a, b, c):
    """Calculate the angle (in degrees) at point b formed by points a-b-c"""
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-8)
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)


def extract_joint_coordinates(frame, pose):
    """Extracts (x, y, z) coordinates of key joints"""
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)
    joints = {}
    if results.pose_landmarks:
        for idx, lm in enumerate(results.pose_landmarks.landmark):
            joints[idx] = (lm.x, lm.y, lm.z)
    return joints


def analyze_angles(joint_data):
    """Calculates elbow and shoulder angles for each frame"""
    angles_list = []
    for joints in joint_data:
        try:
            left_elbow_angle = calculate_angle(joints[11][:2], joints[13][:2], joints[15][:2])
            right_elbow_angle = calculate_angle(joints[12][:2], joints[14][:2], joints[16][:2])
            left_shoulder_angle = calculate_angle(joints[13][:2], joints[11][:2], joints[23][:2])
            right_shoulder_angle = calculate_angle(joints[14][:2], joints[12][:2], joints[24][:2])

            angles_list.append({
                "left_elbow": left_elbow_angle,
                "right_elbow": right_elbow_angle,
                "left_shoulder": left_shoulder_angle,
                "right_shoulder": right_shoulder_angle
            })
        except:
            angles_list.append({
                "left_elbow": None,
                "right_elbow": None,
                "left_shoulder": None,
                "right_shoulder": None
            })
    return pd.DataFrame(angles_list)




def recursive_smooth(A, alpha=0.5):
    """Compute S_1 = A_1; S_n = alpha*A_n + (1-alpha)*S_{n-1}"""
    A = np.asarray(A, dtype=float)
    S = np.full_like(A, np.nan)
    if len(A) == 0:
        return S
    valid_idx = np.where(~np.isnan(A))[0]
    if len(valid_idx) == 0:
        return S
    first = valid_idx[0]
    S[first] = A[first]
    for i in range(first + 1, len(A)):
        if np.isnan(A[i]):
            S[i] = S[i - 1]
        else:
            S[i] = alpha * A[i] + (1 - alpha) * S[i - 1]
    return S


def fatigue_metrics_from_recursive(A, alpha=0.5, rolling_window=10):
    """Compute smoothed sequence + fatigue metrics"""
    A = np.asarray(A, dtype=float)
    S = recursive_smooth(A, alpha=alpha)

    residuals = np.abs(A - S)
    residuals[np.isnan(A)] = np.nan

    first_idx = np.where(~np.isnan(S))[0][0]
    last_idx = np.where(~np.isnan(S))[0][-1]
    baseline_drift = S[last_idx] - S[first_idx]
    mean_abs_res = np.nanmean(residuals)

    res_series = pd.Series(residuals)
    rolling_std = res_series.rolling(window=rolling_window, min_periods=1).std()
    mean_rolling_std = rolling_std.mean()

    try:
        valid_i = np.where(~np.isnan(residuals))[0]
        if len(valid_i) >= 2:
            X = valid_i.reshape(-1, 1)
            y = residuals[valid_i]
            lr = LinearRegression().fit(X, y)
            residual_slope = lr.coef_[0]
        else:
            residual_slope = 0.0
    except Exception:
        residual_slope = 0.0

    # Weighted fatigue score
    w_drift, w_res, w_var = 0.6, 0.8, 0.6
    fatigue_score = (
        w_drift * abs(baseline_drift)
        + w_res * mean_abs_res
        + w_var * (mean_rolling_std if not np.isnan(mean_rolling_std) else 0)
    )

    # Interpret fatigue level
    if fatigue_score > 25:
        fatigue_level = "High"
    elif fatigue_score > 10:
        fatigue_level = "Moderate"
    else:
        fatigue_level = "Low"

    metrics = {
        "baseline_drift": float(baseline_drift),
        "mean_abs_residual": float(mean_abs_res),
        "mean_rolling_std": float(mean_rolling_std) if not np.isnan(mean_rolling_std) else None,
        "residual_slope": float(residual_slope),
        "fatigue_score": float(fatigue_score)
    }

    return {"S": S, "residuals": residuals, "metrics": metrics, "fatigue_level": fatigue_level}




st.title(" Basketball Shooting Form & Fatigue Analyzer (Recursive Equation Method)")

uploaded_video = st.file_uploader("Upload Basketball Shooting Video", type=["mp4", "mov", "avi"])

if uploaded_video:
    # Save video temporarily
    tfile = tempfile.NamedTemporaryFile(delete=False)
    tfile.write(uploaded_video.read())
    video_path = tfile.name

    # Initialize MediaPipe
    mp_drawing = mp.solutions.drawing_utils
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

    cap = cv2.VideoCapture(video_path)
    frame_placeholder = st.empty()
    progress_text = st.empty()
    progress_bar = st.progress(0)

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    joint_data = []
    frame_count = 0

    st.info("Processing video and detecting skeletons...")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(frame_rgb)

        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2)
            )
            joints = {idx: (lm.x, lm.y, lm.z) for idx, lm in enumerate(results.pose_landmarks.landmark)}
            joint_data.append(joints)

        frame_placeholder.image(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB), channels="RGB", use_container_width=True)
        frame_count += 1
        progress_text.text(f"Processing frame {frame_count}/{total_frames}")
        progress_bar.progress(min(frame_count / total_frames, 1.0))

    cap.release()
    os.remove(video_path)
    st.success("✅ Skeleton overlay complete!")


    st.info("Analyzing angles and fatigue...")
    angles_df = analyze_angles(joint_data)
    st.dataframe(angles_df.head())

    # Apply recursive fatigue model to elbows
    left_result = fatigue_metrics_from_recursive(angles_df["left_elbow"].values)
    right_result = fatigue_metrics_from_recursive(angles_df["right_elbow"].values)

    # Combine fatigue scores
    combined_score = (left_result["metrics"]["fatigue_score"] + right_result["metrics"]["fatigue_score"]) / 2
    if combined_score > 25:
        overall_level = "High"
    elif combined_score > 10:
        overall_level = "Moderate"
    else:
        overall_level = "Low"


    st.subheader("Left Elbow Angle vs Smoothed Sequence")
    st.line_chart(pd.DataFrame({
        "Raw Left Elbow": angles_df["left_elbow"],
        "Smoothed (S)": left_result["S"]
    }))

    st.subheader("Right Elbow Angle vs Smoothed Sequence")
    st.line_chart(pd.DataFrame({
        "Raw Right Elbow": angles_df["right_elbow"],
        "Smoothed (S)": right_result["S"]
    }))

    # Show metrics
    st.write("### Left Elbow Metrics")
    st.json(left_result["metrics"])

    st.write("### Right Elbow Metrics")
    st.json(right_result["metrics"])

    # Overall fatigue summary
    st.subheader(f"🏋️‍♂️ Overall Fatigue Level: {overall_level}")
    if overall_level == "Low":
        st.success("Form looks stable — low fatigue detected ✅")
    elif overall_level == "Moderate":
        st.warning("Some drift or inconsistency detected — possible mild fatigue ⚠️")
    else:
        st.error("Significant fatigue detected — form breakdown likely ❌")

    st.write("""
    ### 📈 How it Works
    This uses the recursive equation:
    \[
    S_1 = A_1, \quad S_n = \frac{S_{n-1} + A_n}{2}
    \]
    - **S** is a smoothed baseline angle sequence.
    - **Drift** in S shows gradual form degradation.
    - **Residuals (A − S)** show instability or shakiness.
    - **Variability** in residuals and slope changes indicate fatigue onset.
    """)
else:
    st.info("Upload a basketball shooting video to begin analysis.")
